# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
display(df.head())
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4963
               request       0.76      0.43      0.55      1092
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.62      0.67      2724
          medical_help       0.64      0.10      0.17       564
      medical_products       0.69      0.10      0.17       344
     search_and_rescue       0.56      0.03      0.05       185
              security       0.00      0.00      0.00       127
              military       0.58      0.08      0.15       229
                 water       0.86      0.32      0.47       411
                  food       0.83      0.54      0.66       734
               shelter       0.81      0.38      0.51       554
              clothing       0.73      0.08      0.15        97
                 money       0.73      0.05      0.10       148
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


the f-score for most of message categories above are very low and the that might occured because of the unequal number of messages associated with each category as seen in the support column.

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
    parameters = {
        'clf__estimator__max_depth': [10, 20],
        'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2, 4]
    }

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='f1_micro', verbose = 5, n_jobs=5)

grid_fit = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.3806766781255654, total=  53.2s
[CV] clf__estimator__max_depth=10, clf__estimator_

[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:  3.1min


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.3724097745729998, total= 1.3min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100, score=0.38684507966602255, total= 1.3min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.3877670044470366, total=  51.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_d

[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed: 21.2min


[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.4182946611185557, total= 1.0min
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.4163817714023896, total= 1.5min
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.4216927453769559, total= 1.5min
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_de

[Parallel(n_jobs=5)]: Done  72 out of  72 | elapsed: 24.4min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
print('best f-score achieved:{}'.format(grid_fit.best_score_))
print('best parameters:{}'.format(grid_fit.best_params_))



best f-score achieved:0.4296926774524259
best parameters:{'clf__estimator__max_depth': 20, 'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 50}


In [9]:
grid_fit

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=5,
       param_grid={'clf__estimator__max_depth': [10, 20], 'clf__estimator__min_samples_leaf': [1, 2, 4], 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

In [10]:
best_model = grid_fit.best_estimator_
Y_pred_tuned = best_model.predict(X_test)
print(classification_report(Y_test.values, Y_pred_tuned, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      4963
               request       1.00      0.01      0.03      1092
                 offer       0.00      0.00      0.00        27
           aid_related       0.89      0.29      0.44      2724
          medical_help       0.00      0.00      0.00       564
      medical_products       0.00      0.00      0.00       344
     search_and_rescue       0.00      0.00      0.00       185
              security       0.00      0.00      0.00       127
              military       0.00      0.00      0.00       229
                 water       1.00      0.00      0.00       411
                  food       0.00      0.00      0.00       734
               shelter       1.00      0.01      0.01       554
              clothing       0.00      0.00      0.00        97
                 money       0.00      0.00      0.00       148
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


as it can be seen above, I could not reach high f-score value and that's mainly due to the messages distribution into categories. Some categories have a high number of messages and some don't which will won't allow the model to learn much about the categories.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
filename='classifier.pkl'
pickle.dump(best_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.